In [1]:
import pandas as pd

In [2]:
df = pd.read_json(path_or_buf='gg2013.json')
# df = pd.read_json(path_or_buf='gg2015.json')

In [3]:
data = df['text']

In [4]:
# sample data if necessary
sample_size = 200000
if len(df) > sample_size:
    data = data.sample(n=sample_size)
len(data)

174643

In [5]:
import json
with open('autograder/gg2013answers.json') as file:
    ground_truth = json.load(file)
file.closed

True

In [6]:
true_award_winners = ground_truth['award_data']
awards_old = [item[0] for item in true_award_winners.items()]

In [7]:
presenters_old = [true_award_winners[item[0]]['presenters'] for item in true_award_winners.items()]
presenters_old

[['robert pattinson', 'amanda seyfried'],
 ['halle berry'],
 ['aziz ansari', 'jason bateman'],
 ['arnold schwarzenegger', 'sylvester stallone'],
 ['bradley cooper', 'kate hudson'],
 ['dennis quaid', 'kerry washington'],
 ['dustin hoffman'],
 ['will ferrell', 'kristen wiig'],
 ['don cheadle', 'eva longoria'],
 ['jennifer lopez', 'jason statham'],
 ['nathan fillion', 'lea michele'],
 ['george clooney'],
 ['robert downey, jr.'],
 ['jennifer garner'],
 ['julia roberts'],
 ['kristen bell', 'john krasinski'],
 ['megan fox', 'jonah hill'],
 ['salma hayek', 'paul rudd'],
 ['jessica alba', 'kiefer sutherland'],
 ['don cheadle', 'eva longoria'],
 ['sacha baron cohen'],
 ['jennifer lopez', 'jason statham'],
 ['george clooney'],
 ['jimmy fallon', 'jay leno'],
 ['salma hayek', 'paul rudd'],
 ['lucy liu', 'debra messing']]

In [8]:
# hard-coded awards 2013-2015
awards_old

['best screenplay - motion picture',
 'best director - motion picture',
 'best performance by an actress in a television series - comedy or musical',
 'best foreign language film',
 'best performance by an actor in a supporting role in a motion picture',
 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television',
 'best motion picture - comedy or musical',
 'best performance by an actress in a motion picture - comedy or musical',
 'best mini-series or motion picture made for television',
 'best original score - motion picture',
 'best performance by an actress in a television series - drama',
 'best performance by an actress in a motion picture - drama',
 'cecil b. demille award',
 'best performance by an actor in a motion picture - comedy or musical',
 'best motion picture - drama',
 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television',
 'best performance by an actr

In [9]:
# hard-coded awards for 2018-2019
awards = ['Best Motion Picture - Drama',
         'Best Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Motion Picture - Drama',
         'Best Performance by an Actor in a Motion Picture - Drama',
         'Best Performance by an Actress in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actor in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in any Motion Picture',
         'Best Performance by an Actor in a Supporting Role in any Motion Picture',
         'Best Director - Motion Picture',
         'Best Screenplay - Motion Picture',
         'Best Motion Picture - Animated',
         'Best Motion Picture - Foreign Language',
         'Best Original Score - Motion Picture',
         'Best Original Song - Motion Picture',
         'Best Television Series - Drama',
         'Best Television Series - Musical or Comedy',
         'Best Television Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actress in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actor in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actress In A Television Series - Drama',
         'Best Performance by an Actor In A Television Series - Drama',
         'Best Performance by an Actress in a Television Series - Musical or Comedy',
         'Best Performance by an Actor in a Television Series - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actor in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Cecil B. deMille Award']

In [10]:
# use awards_old and presenters_old
awards = awards_old
presenters = presenters_old

In [11]:
import re

In [12]:
def reduce(line):
    # test
    # replace "television" to "tv"
    pattern = r'\btelevision\b'
    line = re.sub(pattern, 'tv', line.lower())

    # remove words "best", "performance", "film","motion", "picture", "limited", "language", "role", in", "a", "an", "any", "made", "for", "by", "b.", "award", and all punctuations
    pattern = r'\bbest\b|\bperformance\b|\bfilm\b|\bmotion\b|\bpicture\b|\blimited\b|\blanguage\b|\brole\b|\bin\b|\ba\b|\ban\b|\bany\b|\bmade\b|\bfor\b|\bby\b|\bb\b|\baward\b|[^\w\s]'
    return re.sub(pattern, ' ', line.lower()).split()

In [13]:
# test reduce function on old awards
for award in awards_old:
    ls = reduce(award)
    print(sorted(set(ls), key=lambda word: ls.index(word)))

['screenplay']
['director']
['actress', 'tv', 'series', 'comedy', 'or', 'musical']
['foreign']
['actor', 'supporting']
['actress', 'supporting', 'series', 'mini', 'or', 'tv']
['comedy', 'or', 'musical']
['actress', 'comedy', 'or', 'musical']
['mini', 'series', 'or', 'tv']
['original', 'score']
['actress', 'tv', 'series', 'drama']
['actress', 'drama']
['cecil', 'demille']
['actor', 'comedy', 'or', 'musical']
['drama']
['actor', 'supporting', 'series', 'mini', 'or', 'tv']
['actress', 'supporting']
['tv', 'series', 'drama']
['actor', 'mini', 'series', 'or', 'tv']
['actress', 'mini', 'series', 'or', 'tv']
['animated', 'feature']
['original', 'song']
['actor', 'drama']
['tv', 'series', 'comedy', 'or', 'musical']
['actor', 'tv', 'series', 'drama']
['actor', 'tv', 'series', 'comedy', 'or', 'musical']


In [14]:
def generate_award_num_keywords_map(awards):
    awards_reduced = []
    award_num_keywords_map = {}
    for i, award in enumerate([sorted(set(reduce(award)), key=lambda word: reduce(award).index(word)) for award in awards]):
        if 'or' in award:
            # find number of words from "or" to the end of award name
            num_words = len(award) - 1 - award.index('or')
            award.remove('or')
            # do not count words from "or" to the end of award name
            award_num_keywords_map[i] = len(award) - num_words 
        else:
            award_num_keywords_map[i] = len(award)
        awards_reduced.append(award)
    return award_num_keywords_map, awards_reduced

award_num_keywords_map, awards_reduced = generate_award_num_keywords_map(awards)

In [15]:
awards_reduced

[['screenplay'],
 ['director'],
 ['actress', 'tv', 'series', 'comedy', 'musical'],
 ['foreign'],
 ['actor', 'supporting'],
 ['actress', 'supporting', 'series', 'mini', 'tv'],
 ['comedy', 'musical'],
 ['actress', 'comedy', 'musical'],
 ['mini', 'series', 'tv'],
 ['original', 'score'],
 ['actress', 'tv', 'series', 'drama'],
 ['actress', 'drama'],
 ['cecil', 'demille'],
 ['actor', 'comedy', 'musical'],
 ['drama'],
 ['actor', 'supporting', 'series', 'mini', 'tv'],
 ['actress', 'supporting'],
 ['tv', 'series', 'drama'],
 ['actor', 'mini', 'series', 'tv'],
 ['actress', 'mini', 'series', 'tv'],
 ['animated', 'feature'],
 ['original', 'song'],
 ['actor', 'drama'],
 ['tv', 'series', 'comedy', 'musical'],
 ['actor', 'tv', 'series', 'drama'],
 ['actor', 'tv', 'series', 'comedy', 'musical']]

In [16]:
def describe_awards(awards, awards_reduced, award_num_keywords_map):
    pairs = sorted(award_num_keywords_map.items(), key=lambda pair: pair[0])
    for pair in pairs:
        print(pair[0])
        print(awards[pair[0]])
        print(awards_reduced[pair[0]])
        if pair[1] < len(awards_reduced[pair[0]]):
            print('******')
        print(pair) # (index, num_keywords_to_match)    
        print()
        
describe_awards(awards, awards_reduced, award_num_keywords_map)

0
best screenplay - motion picture
['screenplay']
(0, 1)

1
best director - motion picture
['director']
(1, 1)

2
best performance by an actress in a television series - comedy or musical
['actress', 'tv', 'series', 'comedy', 'musical']
******
(2, 4)

3
best foreign language film
['foreign']
(3, 1)

4
best performance by an actor in a supporting role in a motion picture
['actor', 'supporting']
(4, 2)

5
best performance by an actress in a supporting role in a series, mini-series or motion picture made for television
['actress', 'supporting', 'series', 'mini', 'tv']
******
(5, 4)

6
best motion picture - comedy or musical
['comedy', 'musical']
******
(6, 1)

7
best performance by an actress in a motion picture - comedy or musical
['actress', 'comedy', 'musical']
******
(7, 2)

8
best mini-series or motion picture made for television
['mini', 'series', 'tv']
******
(8, 2)

9
best original score - motion picture
['original', 'score']
(9, 2)

10
best performance by an actress in a televisi

In [17]:
# tweaking magic
# awards_reduced[0].append('movie')
# awards_reduced[1].append('movie')
# awards_reduced[2].append('movie')
# awards_reduced[11].append('language')

# describe_awards(awards, awards_reduced, award_num_keywords_map)

In [18]:
retweets_freq_dict = {}

def remove_retweet_prefix(line):
    # find 'RT @abc: ' where abc's length is arbitrary
    pattern = re.compile(r'\bRT @([\w\'/]*)\b: ') 
    match = re.search(pattern, line)
    if match:
        # store corresponding retweet without 'RT @' prefix
        string = match.group()[4:]
        if string in retweets_freq_dict:
            retweets_freq_dict[string] += 1
        else:
            retweets_freq_dict[string] = 1
    return re.sub(pattern, ' ', line)

In [19]:
hashtag_freq_dict = {}

def remove_hashtag(line):
    pattern = re.compile(r'#([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in hashtag_freq_dict:
                hashtag_freq_dict[match] += 1
            else:
                hashtag_freq_dict[match] = 1
    return re.sub(pattern, ' ', line)

In [20]:
at_freq_dict = {}

def remove_at(line):
    pattern = re.compile(r'@([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        for match in matches:
            if match in at_freq_dict:
                at_freq_dict[match] += 1
            else:
                at_freq_dict[match] = 1
    return re.sub(pattern, ' ', line)

In [21]:
def remove_url(line):
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b')
    return re.sub(pattern, ' ', line)

In [22]:
def cleanse(line):
    # replace everything to ' ' except alphanumeric character, whitespace, apostrophe, hashtag
    return re.sub(r'[^\w\s\'#]', ' ', line)

In [23]:
cleansed_data = []
for tweet in data:
    line = remove_retweet_prefix(tweet)
    line = remove_hashtag(line) # consider not removing this
    line = remove_at(line) # consider not removing this
    line = remove_url(line)
    line = cleanse(line)
    cleansed_data.append(line)

In [24]:
import spacy
# python3 -m spacy download en
nlp = spacy.load('en')

In [25]:
def identify_entities(text):
    entities = list(nlp(text).ents)
    tags = {}
    for entity in entities:
        if entity not in tags:
            tags[' '.join(t.orth_ for t in entity).strip()]=[entity.label_]
    return tags

In [26]:
# top retweets (each as an entity)
sorted(retweets_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:50]

[('goldenglobes: ', 3459),
 ('eonline: ', 2891),
 ('PerezHilton: ', 2301),
 ('TheEllenShow: ', 1696),
 ('EmWatson: ', 1147),
 ('VanityFair: ', 820),
 ('nbcsnl: ', 743),
 ('CNNshowbiz: ', 699),
 ('CiudadBizarra: ', 510),
 ('BuzzFeed: ', 491),
 ('EW: ', 485),
 ('nbc: ', 465),
 ('vulture: ', 446),
 ('piersmorgan: ', 441),
 ('MARLONLWAYANS: ', 384),
 ('HuffingtonPost: ', 369),
 ('buckhollywood: ', 353),
 ('MarilynMonroeES: ', 334),
 ('TVGuide: ', 319),
 ('THR: ', 315),
 ('DavidSpade: ', 312),
 ('MTVNews: ', 307),
 ('PimpBillClinton: ', 281),
 ('cinema21: ', 281),
 ('washingtonpost: ', 275),
 ('ninagarcia: ', 258),
 ('Cosmopolitan: ', 239),
 ('peopleenespanol: ', 219),
 ('RichardCrouse: ', 217),
 ('peoplemag: ', 213),
 ('prodigalsam: ', 212),
 ('kumailn: ', 211),
 ('DannyZuker: ', 207),
 ('jianghomeshi: ', 201),
 ('HuffPostWomen: ', 200),
 ('DamienFahey: ', 200),
 ('girlsHBO: ', 198),
 ('MHarrisPerry: ', 194),
 ('DougBenson: ', 192),
 ('rogergzz: ', 191),
 ('InStyle: ', 191),
 ('EonlineLati

In [27]:
# top hashtags
sorted(hashtag_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:50]

[('GoldenGlobes', 110135),
 ('goldenglobes', 33113),
 ('Argo', 842),
 ('GetGlue', 769),
 ('Homeland', 696),
 ('redcarpet', 675),
 ('GoldenGlobe', 571),
 ('Goldenglobes', 560),
 ('JodieFoster', 486),
 ('Girls', 451),
 ('killingit', 447),
 ('GOLDENGLOBES', 420),
 ('AlfombraRojaE', 385),
 ('Skyfall', 360),
 ('GlobosdeOro', 352),
 ('RedCarpet', 348),
 ('Lincoln', 348),
 ('LesMis', 346),
 ('GIRLS', 332),
 ('Adele', 289),
 ('LesMiserables', 282),
 ('JenniferLawrence', 258),
 ('GG2013', 254),
 ('homeland', 251),
 ('GoldenGlobes2013', 249),
 ('TinaFey', 226),
 ('ERedCarpet', 224),
 ('AmyPoehler', 209),
 ('eredcarpet', 207),
 ('jodiefoster', 204),
 ('DowntonAbbey', 193),
 ('AnneHathaway', 191),
 ('girls', 188),
 ('DjangoUnchained', 184),
 ('Oscars', 176),
 ('LoveThoseLadies', 163),
 ('Django', 158),
 ('BenAffleck', 154),
 ('Globes', 149),
 ('GameChange', 146),
 ('LOVEHER', 146),
 ('goldenGlobes', 143),
 ('fb', 143),
 ('BillClinton', 142),
 ('fashion', 138),
 ('Oscar', 138),
 ('skyfall', 137),
 

In [28]:
# top @
sorted(at_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:50]

[('goldenglobes', 3150),
 ('OfficialAdele', 1122),
 ('lenadunham', 980),
 ('GoldenGlobes', 863),
 ('BenAffleck', 526),
 ('RealHughJackman', 386),
 ('PerezHilton', 361),
 ('SHO_Homeland', 357),
 ('girlsHBO', 352),
 ('SofiaVergara', 303),
 ('eonline', 290),
 ('taylorswift13', 270),
 ('TNTLA', 255),
 ('JLo', 235),
 ('PaulEpworth', 223),
 ('GirlsHBO', 182),
 ('msleamichele', 167),
 ('SelenaGomez', 164),
 ('VanessuHudgens', 163),
 ('LenaDunham', 156),
 ('steph_hart', 156),
 ('RyanGosling', 137),
 ('kerrywashington', 125),
 ('PixarBrave', 120),
 ('LesMiserables', 113),
 ('nbc', 112),
 ('Chanel', 112),
 ('NathanFillion', 105),
 ('KevalBaxi', 97),
 ('Burberry', 86),
 ('Lewis_Damian', 86),
 ('jessicaalba', 84),
 ('CHANEL', 83),
 ('tomandlorenzo', 78),
 ('nbcsnl', 77),
 ('iamdoncheadle', 76),
 ('piersmorgan', 69),
 ('CiudadBizarra', 67),
 ('CNNshowbiz', 64),
 ('BuzzFeed', 64),
 ('LeoDiCaprio', 62),
 ('EvaLongoria', 61),
 ('azizansari', 60),
 ('gabrielledoug', 60),
 ('Sarah_Hyland', 60),
 ("lenad

In [29]:
from fuzzywuzzy import fuzz

def remove_goldeb_globes(top_results, entity_freq_dict):
    golden_globes = [name for name in [pair[0] for pair in top_results] if fuzz.ratio(name.lower(), 'golden globes') > 60]
    for name in golden_globes:
        if name in entity_freq_dict:
            del entity_freq_dict[name]
    return entity_freq_dict


# we also consider dropping all lower cases examples or examples that contain digit(s), which are not names
def filter_names(pair_list, entity_freq_dict):
    filtered_results = []
    for pair in pair_list:
        string = ''.join(pair[0].split())
        if not all(char.islower() for char in string) and not any(char.isdigit() for char in string):
            filtered_results.append(pair)
        else:
            if pair[0] in entity_freq_dict:
                del entity_freq_dict[pair[0]]
    return filtered_results, entity_freq_dict


def merge_names(top_results, entity_freq_dict):
    names = [pair[0] for pair in top_results]
    names_clusters = []
    for name in names:
        # each name starts as a cluster
        cluster = [name]
        names_to_reduce = names[:]
        names_to_reduce.remove(name)
        # one vs. all comparisons
        for i in names_to_reduce:
            ratio = fuzz.ratio(name.lower(), i.lower())
            # if similarity is larger than 75 or one name is contained in the other name
            if ratio > 75 or re.search(name, i, flags=re.IGNORECASE) or re.search(i, name, flags=re.IGNORECASE):
                cluster.append(i)
        # if multiple names are identified in one cluster
        if len(cluster) > 1:
            names_clusters.append(cluster)

    # find names clusters that should merge
    # ['Amy Poehler', 'Amy', 'Amy Poelher']
    # ['Tina', 'Tina Fey']

    # sort clusters
    names_clusters.sort()
    # sort within each cluster
    names_clusters = ['|'.join(sorted(cluster)) for cluster in names_clusters]
    # remove overlaps
    names_clusters_reduced = [line.split('|') for line in list(set(names_clusters))]
    # sort by length from shortest to longest (merge from the shortest)
    names_clusters_reduced.sort(key=len)

    # weighted frequency of an entity is defined by its frequency multiplied by its string length
    def weighted_freq(element):
        return entity_freq_dict[element] * len(element)

    e = entity_freq_dict.copy()
    for cluster in names_clusters_reduced:
        # select the entity name with highest weighted frequency
        selected_entity_name = max(cluster, key=weighted_freq)
        cluster.remove(selected_entity_name)
        # for names to be merged to the selected entity name
        for name in cluster:
            # if not deleted in previous cases, cumulate frequencies to the selected entity
            if name in e and selected_entity_name in e:
                e[selected_entity_name] += e[name]
                # reward merging
                e[selected_entity_name] += 2
                del e[name]

    top_10 = sorted(e.items(), key=lambda pair: pair[1], reverse=True)[:10]
    return top_10

In [30]:
%%time
# test
num = 0
count = 0
pattern = re.compile(r'\bpresenter|\bpresent', re.IGNORECASE)
for line in cleansed_data:
    match = re.findall(pattern, line.lower())                  
    if match:
        tags = identify_entities(line)
        count += 1
        # check the first 10 cases
        if count < 10:
            print(num)
            print(match)
            print(line)
            print(tags.keys())
            print()
    num += 1
    
print(count)

48
['present']
Definetely Kate's top designers are present in the   tonight of course thanks to the  
dict_keys(['Kate', 'tonight'])

465
['present']
    comentando los     mejores presentadores no hay  
dict_keys([])

1176
['present']
     comentando los     mejores presentadores no hay  
dict_keys(['', 'los'])

1561
['presenter']
 What's your verdict on   presenter Giuliana Rancic's Oriett Domenech dress     
dict_keys(["Giuliana Rancic 's", 'Oriett Domenech'])

1976
['presenter']
THISSSSSSSSSS RT   It's appalling that Kerry Washington is there only as a presenter and not as a nominee     
dict_keys(['THISSSSSSSSSS RT', 'Kerry Washington'])

3247
['presenter']
Jay Leno a presenter      
dict_keys(['Jay Leno', ''])

3451
['presenter']
 Presenter  amp  Oscar Award winner Halle Berry is showstopping in a printed gown from Atelier Versace Fall 2012 at the Golden Globes right now 
dict_keys(['Presenter', 'Oscar Award', 'Halle Berry', 'Atelier Versace', 'Fall 2012', 'the Golden Globes'])



In [31]:
def check_target_words(awards_reduced):
    target_words = ['actor', 'actress', 'director', 'score', 'song', 'foreign', 'tv']
    for word in target_words:
        if word in awards_reduced:
            return word
    return False

def check_primary_target_words(awards_reduced):
    target_words = ['actor', 'actress', 'director', 'score', 'song']
    for word in target_words:
        if word in awards_reduced:
            return word
    return False

check_target_words(awards_reduced[12])

False

In [32]:
def find_award_presenter(awards, awards_reduced, award_index, verbose=False):
    print(award_index)
    print('Predicting for:', awards[award_index])
    print('true presenter:', presenters[award_index])
    award = awards_reduced[award_index]
    print('award name reduced:', award)
    num_keywords_to_match = award_num_keywords_map[award_index]
    print("num keywords to match:", num_keywords_to_match)
    
    # add word boundary '\b' to prevent grabbing examples like "showing" and "wonder"    
    pattern = re.compile(r'\bpresenter|\bpresent', re.IGNORECASE)
    pattern2 = re.compile(r'\bbest\b', re.IGNORECASE)
        
    entity_dict = {}
    
    num = 0
    flag = 0
    target_word = check_target_words(award)
    
    if len(award) != num_keywords_to_match:
        flag = 1
        # find target word pattern to match for sure
#         primary_target_word = check_primary_target_words(award)  
#         if primary_target_word:
#             target_word_pattern = re.compile(r'\b{0}\b'.format(primary_target_word), re.IGNORECASE)
#             print("matching target word '{0}'".format(target_word))
#         # else if primary keyword is not found, match for secondary keyword
#         elif 'tv' in award:
#             target_word_pattern = re.compile(r'\b{0}\b'.format('tv'), re.IGNORECASE)
#             print("matching target word 'tv'")
#         else:
#             target_word_pattern = re.compile(r'\b{0}\b'.format(award[0]), re.IGNORECASE)
#             print("matching target word '{0}'".format(award[0]))
                
        for line in cleansed_data:
            match = re.findall(pattern, line.lower())
#             match_best = re.findall(pattern2, line.lower())
            weight = 20 if any('presenter' in tup for tup in match) or any('presenters' in tup for tup in match) else 1
            
#             match_target_word = re.findall(target_word_pattern, line.lower())
            num_keywords_matched = len(set(award).intersection(set(line.split())))
            
            # if line contains at least number keywords to match and pattern is found
#             if match and num_keywords_matched >= num_keywords_to_match and match_target_word:
            if match and num_keywords_matched >= num_keywords_to_match:
        
                ratio = round(num_keywords_matched/(num_keywords_to_match+0.0001))
                weight *= ratio
                
                tags = identify_entities(line)

                if verbose:
                    # print the first 5 occurrences
                    if num < 5:
                        print(match)
                        print(line)
                        print(tags.keys())
                        print()

                for entity in tags.keys():
                    entity_raw = entity
                    entity = entity.strip() 
                    if len(entity) > 1:
                        # add more weights for appropriate entity classification
                        if tags[entity_raw] == 'PERSON':
                            weight += 5
                        if tags[entity_raw] == 'PERSON':
                            weight -= 5 
                        if entity not in entity_dict:
                            entity_dict[entity] = weight
                        else:
                            entity_dict[entity] += weight
                num += 1
    
    else:
        for line in cleansed_data:
            match = re.findall(pattern, line.lower())
            weight = 20 if any('presenter' in tup for tup in match) or any('presenters' in tup for tup in match) else 1
            
            # if line contains at least number keywords to match and pattern is found
            num_keywords_matched = len(set(award).intersection(set(line.split())))
            if num_keywords_matched == num_keywords_to_match and match:
                
                ratio = round(num_keywords_matched/(num_keywords_to_match+0.0001))
                weight *= ratio
                tags = identify_entities(line)

                if verbose:
                    # print the first 5 occurrences
                    if num < 5:
                        print(match)
                        print(any('presenter' in tup for tup in match))
                        print(line)
                        print(tags.keys())
                        print()

                for entity in tags.keys():
                    entity_raw = entity
                    entity = entity.strip() 
                    if len(entity) > 1:  
                        # add more weights for appropriate entity classification
                        if tags[entity_raw] == 'PERSON':
                            weight += 5
                        if tags[entity_raw] == 'PERSON':
                            weight -= 5
                        if entity not in entity_dict:
                            entity_dict[entity] = weight
                        else:
                            entity_dict[entity] += weight
                num += 1
    
    # if no results found, recursively reducing num_keywords_to_match
    while(num == 0 or num < 10):
        print('no results found or too few matches!!! add more keywords and reduce num_keywords_to_match!!!')
                
        num_keywords_to_match -= 1
        print("num keywords to match:", num_keywords_to_match)
        if num_keywords_to_match == 0:
            print('break loop!!!')
            break
        
        print('award before', award)
        # add alternative word for 'tv'
        if 'tv' in award and 'television' not in award:
            award.append('television')
        # add alternative word for 'Motion Picture'
        if 'Motion Picture' in awards[award_index]:
            award.append('movie')
            award.append('motion')
            award.append('picture')
        print('expanded award with more keywards', award)
            
        if flag == 1:       
#             print("matching target word '{0}'".format(target_word))
            for line in cleansed_data:
                match = re.findall(pattern, line.lower())
                weight = 20 if any('presenter' in tup for tup in match) or any('presenters' in tup for tup in match) else 1
                
#                 match_target_word = re.findall(target_word_pattern, line.lower())
                num_keywords_matched = len(set(award).intersection(set(line.split())))
                
                # if line contains at least number keywords to match and pattern is found
#                 if match and num_keywords_matched >= num_keywords_to_match and match_target_word:
                if match and num_keywords_matched >= num_keywords_to_match:
            
                    ratio = round(num_keywords_matched/(num_keywords_to_match+0.0001))
                    weight *= ratio
                    tags = identify_entities(line)

                    if verbose:
                        # print the first 5 occurrences
                        if num < 5:
                            print(match)
                            print(line)
                            print(tags.keys())
                            print()

                    for entity in tags.keys():
                        entity_raw = entity
                        entity = entity.strip() 
                        if len(entity) > 1: 
                            # add more weights for appropriate entity classification
                            if tags[entity_raw] == 'PERSON':
                                weight += 5
                            if tags[entity_raw] == 'PERSON':
                                weight -= 5
                            if entity not in entity_dict:
                                entity_dict[entity] = weight
                            else:
                                entity_dict[entity] += weight
                    num += 1
        else:
            for line in cleansed_data:
                match = re.findall(pattern, line.lower())
                weight = 20 if any('presenter' in tup for tup in match) or any('presenters' in tup for tup in match) else 1
                
                # if line contains at least number keywords to match and pattern is found
                num_keywords_matched = len(set(award).intersection(set(line.split())))
                if num_keywords_matched == num_keywords_to_match and match:
                    
                    ratio = round(num_keywords_matched/(num_keywords_to_match+0.0001))
                    weight *= ratio
                    tags = identify_entities(line)

                    if verbose:
                        # print the first 5 occurrences
                        if num < 5:
                            print(match)
                            print(line)
                            print(tags.keys())
                            print()

                    for entity in tags.keys():
                        entity_raw = entity
                        entity = entity.strip() 
                        if len(entity) > 1: 
                            # add more weights for appropriate entity classification
                            if tags[entity_raw] == 'PERSON':
                                weight += 5
                            if tags[entity_raw] == 'PERSON':
                                weight -= 5
                            if entity not in entity_dict:
                                entity_dict[entity] = weight
                            else:
                                entity_dict[entity] += weight
                    num += 1
    print('num of matches:', num)
    return entity_dict

# Predictions for Award Presenters

In [33]:
%%time

# test by changing key
key = 15
awards_reduced_copy = awards_reduced[:]

entity_freq_dict = find_award_presenter(awards, awards_reduced_copy, key, verbose=False)
top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:100]
# remove 'golden globes' from identified host names
entity_freq_dict = remove_goldeb_globes(top_results, entity_freq_dict)
top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:30]
# filter for names if necessary
if 'actor' in awards_reduced[key] or 'actress' in awards_reduced[key] or 'director' in awards_reduced[key]:
    top_results, entity_freq_dict = filter_names(top_results, entity_freq_dict)
print('top results:')
print(top_results)
print('top results after merging:')
top_10 = merge_names(top_results, entity_freq_dict)
print(top_10)

15
Predicting for: best performance by an actor in a supporting role in a series, mini-series or motion picture made for television
true presenter: ['kristen bell', 'john krasinski']
award name reduced: ['actor', 'supporting', 'series', 'mini', 'tv']
num keywords to match: 4
no results found or too few matches!!! add more keywords and reduce num_keywords_to_match!!!
num keywords to match: 3
award before ['actor', 'supporting', 'series', 'mini', 'tv']
expanded award with more keywards ['actor', 'supporting', 'series', 'mini', 'tv', 'television']
no results found or too few matches!!! add more keywords and reduce num_keywords_to_match!!!
num keywords to match: 2
award before ['actor', 'supporting', 'series', 'mini', 'tv', 'television']
expanded award with more keywards ['actor', 'supporting', 'series', 'mini', 'tv', 'television']
num of matches: 25
top results:
[('Maggie Smith', 21), ('Downton Abbey', 21), ('Dennis Quaid', 4), ('Kerry Washington', 4), ('Mini', 1), ('Amazing', 1)]
top res

In [34]:
%%time
award_presenters_dict = {}

for key in award_num_keywords_map.keys():
    entity_freq_dict = find_award_presenter(awards, awards_reduced, key, verbose=False)
    top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:100]
    # remove 'golden globes' from identified host names
    entity_freq_dict = remove_goldeb_globes(top_results, entity_freq_dict)
    top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:30]
    # filter for names if necessary
    top_results, entity_freq_dict = filter_names(top_results, entity_freq_dict)
    print('top results:')
    print(top_results)
    print('top results after merging:')
    top_10 = merge_names(top_results, entity_freq_dict)
    print(top_10)
    
    if len(top_10) != 0:
        award_presenters_dict[awards[key].lower()] = top_10[0][0]
    else:
        award_presenters_dict[awards[key].lower()] = ''
    print()

print(award_winner_dict)

0
Predicting for: best screenplay - motion picture
true presenter: ['robert pattinson', 'amanda seyfried']
award name reduced: ['screenplay']
num keywords to match: 1
no results found or too few matches!!! add more keywords and reduce num_keywords_to_match!!!
num keywords to match: 0
break loop!!!
num of matches: 6
top results:
[('Robert Pattinson', 3), ('Amanda Seyfried', 1), ("Robert Pattinson 's", 1)]
top results after merging:
[('Robert Pattinson', 6), ('Amanda Seyfried', 1)]

1
Predicting for: best director - motion picture
true presenter: ['halle berry']
award name reduced: ['director']
num keywords to match: 1
no results found or too few matches!!! add more keywords and reduce num_keywords_to_match!!!
num keywords to match: 0
break loop!!!
num of matches: 9
top results:
[('Ben Affleck', 62), ('Jennifer Garner', 60), ('Halle Berry', 5), ('El', 2)]
top results after merging:
[('Ben Affleck', 62), ('Jennifer Garner', 60), ('Halle Berry', 5), ('El', 2)]

2
Predicting for: best perfo

no results found or too few matches!!! add more keywords and reduce num_keywords_to_match!!!
num keywords to match: 0
break loop!!!
num of matches: 0
top results:
[]
top results after merging:
[]

10
Predicting for: best performance by an actress in a television series - drama
true presenter: ['nathan fillion', 'lea michele']
award name reduced: ['actress', 'tv', 'series', 'drama']
num keywords to match: 4
no results found or too few matches!!! add more keywords and reduce num_keywords_to_match!!!
num keywords to match: 3
award before ['actress', 'tv', 'series', 'drama']
expanded award with more keywards ['actress', 'tv', 'series', 'drama', 'television']
no results found or too few matches!!! add more keywords and reduce num_keywords_to_match!!!
num keywords to match: 2
award before ['actress', 'tv', 'series', 'drama', 'television']
expanded award with more keywards ['actress', 'tv', 'series', 'drama', 'television']
num of matches: 24
top results:
[('Maggie Smith', 21), ('Downton Abbey

num of matches: 47
top results:
[('Iron Man', 40), ('Tina Fey', 40), ('Maggie Smith', 21), ('Downton Abbey', 21), ('Paul Rudd', 20), ('Clinton', 20), ('Proves', 20), ('Dennis Quaid', 5), ('Kerry Washington', 5), ('Mini', 3), ('Amazing', 3), ('Bienvenida', 2), ('Kate Hudson', 1), ('Damian Lewis', 1), ('Homeland', 1), ('Lo', 1), ('Salma Hayek', 1), ('George W', 1), ('Bush', 1), ('U S', 1), ('One', 1), ('Jessicas', 1), ('Sly Stallone', 1), ('Ah nold', 1), ('Schwarzenegger', 1), ('El', 1)]
top results after merging:
[('Iron Man', 40), ('Tina Fey', 40), ('Maggie Smith', 21), ('Downton Abbey', 21), ('Paul Rudd', 20), ('Clinton', 20), ('Proves', 20), ('Sly Stallone', 7), ('Dennis Quaid', 5), ('Kerry Washington', 5)]

19
Predicting for: best performance by an actress in a mini-series or motion picture made for television
true presenter: ['don cheadle', 'eva longoria']
award name reduced: ['actress', 'mini', 'series', 'tv']
num keywords to match: 3
no results found or too few matches!!! add mor

num of matches: 60
top results:
[('Iron Man', 40), ('Tina Fey', 40), ('Kristen Wiig', 32), ('Maggie Smith', 21), ('Downton Abbey', 21), ('Paul Rudd', 20), ('Clinton', 20), ('Proves', 20), ('Will Farrell', 20), ('Presenters', 20), ('Will Ferrell', 12), ('Comedians', 9), ('Jay Leno', 5), ('Jimmy Fallon', 5), ('Dennis Quaid', 3), ('Kerry Washington', 3), ('Mejor', 2), ('Cine', 2), ('Kate Hudson', 1), ('Catherine Zeta Jones', 1), ('al Globo de Oro', 1), ('Rosario Dawson', 1), ('The Best Exotic Marigold Hotel', 1), ('Damian Lewis', 1), ('Homeland', 1), ('Lo', 1), ('Salma Hayek', 1), ('Mini', 1)]
top results after merging:
[('Iron Man', 40), ('Tina Fey', 40), ('Will Farrell', 34), ('Kristen Wiig', 32), ('Maggie Smith', 21), ('Downton Abbey', 21), ('Paul Rudd', 20), ('Clinton', 20), ('Proves', 20), ('Presenters', 20)]

CPU times: user 2min 42s, sys: 17.1 s, total: 2min 59s
Wall time: 59 s
